In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import os
import tensorflow as tf
import numpy as np
from datetime import datetime
from PIL import Image



# line added to solve a warning that tensorflow gave me
tflogger = tf.get_logger()
tflogger.setLevel('ERROR')
# Just disables the warning, doesn't enable AVX/FMA
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set the seed for random operations.
# This let our experiments to be reproducible.
SEED = 1234
tf.random.set_seed(SEED)

# Get current working directory
cwd = os.getcwd() #cwd= /kaggle/working
print(cwd)
# Set GPU memory growth
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation =True 

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1. / 255,
                                        validation_split=0.2)  # fraction of images reserved for validation
else:
    train_data_gen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)

# Create test ImageDataGenerator objects (for validation I train_data_gen, since all the images are in the same directory)
test_data_gen = ImageDataGenerator(rescale=1. / 255)

# Create generators to read images from dataset directory
# -------------------------------------------------------
# dataset_dir = /kaggle/input/Classification_Dataset
dataset_dir = os.path.join('/kaggle', 'input')
dataset_dir = os.path.join(dataset_dir, 'classification-dataset/Classification_Dataset')
print(dataset_dir)
# Batch size
# TODO: ci sono 1554 immagini. 1554 non è divisibile per 8. Va bene se metto 7?
bs = 7

#TODO : probabilmente queste sono da levare
# img shape
img_h = 256
img_w = 256

num_classes = 20

decide_class_indices = True
if decide_class_indices:
    classes = ['owl',               # 0
               'galaxy',            # 1
               'lightning',         # 2
               'wine-bottle',       # 3
               't-shirt',           # 4
               'waterfall',         # 5
               'sword',             # 6
               'school-bus',        # 7
               'calculator',        # 8
               'sheet-music',       # 9
               'airplanes',         # 10
               'lightbulb',         # 11
               'skyscraper',        # 12
               'mountain-bike',     # 13
               'fireworks',         # 14
               'computer-monitor',  # 15
               'bear',              # 16
               'grand-piano',       # 17
               'kangaroo',          # 18
               'laptop']            # 19
else:
    classes = None

# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED,
                                               subset='training')  # set as training data
# Validation
valid_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED,
                                               subset='validation')  # set as validation data

# Test
test_dir = os.path.join(dataset_dir, 'test')
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             batch_size=bs,
                                             classes=classes,
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)

# Create Dataset objects
# ----------------------

# Training
# create a training set whose elements are generated by the generator train_gen
# TODO: visto che le immagini sono di dimensione diversa in output_shapes ho messo None al posto che img_h e img_w
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes])) #TODO: controllare output_shapes
# Shuffle (Already done in generator..)
# train_dataset = train_dataset.shuffle(buffer_size=len(train_gen))

# Normalize images (Already done in generator..)
# def normalize_img(x_, y_):
#     return tf.cast(x_, tf.float32) / 255., y_

# train_dataset = train_dataset.map(normalize_img)

# 1-hot encoding <- for categorical cross entropy (Already done in generator..)
# def to_categorical(x_, y_):
#     return x_, tf.one_hot(y_, depth=10)

# train_dataset = train_dataset.map(to_categorical)

# Divide in batches (Already done in generator..)
# train_dataset = train_dataset.batch(bs)

# Repeat
# Without calling the repeat function the dataset
# will be empty after consuming all the images
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()
# Test
# ----
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
test_dataset = valid_dataset.repeat()

In [ ]:
train_gen.class_indices

In [ ]:
# Create Model
# ------------
model = tf.keras.Sequential()
#add model layers
model.add(tf.keras.layers.Conv2D(20, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(img_h,img_w,3)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Conv2D(40, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Conv2D(60, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Conv2D(80, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(70, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(tf.keras.layers.Dense(50, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(tf.keras.layers.Dense(30, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(tf.keras.layers.Dense(20, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))

In [ ]:
# Visualize created model as a table
model.summary()
# Visualize initialized weights
print(model.weights)

In [ ]:
rm -r /kaggle/working/classification_experiments

In [ ]:
rm /kaggle/working/results_*

In [ ]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'),
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)

# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen),
          callbacks=callbacks)

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser


In [ ]:
#model.load_weights("/kaggle/input")  # use this if you want to restore saved model
#model.load_weights(latest)
eval_out = model.evaluate(x=valid_dataset,
                          steps=len(valid_gen),
                          verbose=0)

print(eval_out)

In [ ]:
def create_csv(results, results_dir='./'):


    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


In [ ]:
from skimage import data, color
from skimage.transform import  resize
from skimage.io import imread
from skimage.io import imshow
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
image_filenames = next(os.walk(test_dir))[2]
results = {}
for image_name in tqdm(image_filenames):
   #img = Image.open(os.path.join(test_dir,image_name)).convert('RGB')
   img = imread(os.path.join(test_dir,image_name))
   img = color.gray2rgb(img)
   img = resize(img, (img_h, img_w))
   imshow(img)
   plt.show()
   img_array = np.array(img) 
   img_array = np.expand_dims(img_array, 0)
   out_softmax = model.predict(x=img_array)
   print("out_softmax:")
   print (out_softmax)
   prediction = tf.math.argmax(out_softmax, axis=1)   # predicted class
   print("prediction:")
   print(prediction[0].numpy())
   results[image_name] = prediction[0].numpy()

create_csv(results)